In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import plotly.express as px
from src.download import clustering_data, bids_data

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import statsmodels.api as sm

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import statsmodels.api as sm

### Cluster

In [165]:
start_date='2025-02-01'
stop_date='2025-02-28'
min_trips_threshold = 300000

In [33]:
df = clustering_data(start_date=start_date,
                     stop_date=stop_date,
                     min_trips_threshold=min_trips_threshold,
                     print_query=False)

df.to_csv(f'data/ClusterCities_{start_date}_{stop_date}.csv', index=False)

In [166]:
df = pd.read_csv(f'data/ClusterCities_{start_date}_{stop_date}.csv')

In [ ]:
def cluster_plot(df):
    # 2. Используем новую колонку для параметра 'symbol'
    fig = px.scatter(
        df,
        x='order2bid',
        y='bid2done',
        # size='total_orders',
        color='order_type',
        symbol='highlight_group',  # Используем новую колонку для символов
        symbol_map={ # Опционально: задаем конкретные символы
            'Highlighted': 'star',
            'Normal': 'circle'
        },
        hover_name='city_id',
        hover_data={
            'city_id': True,
            'order_type': True,
            'total_orders': ':,',
            'total_rides': ':,',
            'bid2done': ':.4f',
            'order2bid': ':.4f',
            'highlight_group': True # Можно добавить в hover_data для ясности
        },
        title=f'Order2Bid (MR) vs Bid2Done (with Highlighted Cities), {start_date} - {stop_date}',
        labels={
            'bid2done': 'Bid2Done',
            'order2bid': 'Order2Bid',
            'total_orders': 'Total Orders',
            'total_rides': 'Total Rides',
            'order_type': 'Order Type',
            'city_id': 'City ID',
            'highlight_group': 'Highlight Status' # Лейбл для новой легенды символов
        }
    )

    fig.update_layout(
        legend_title_text='Order Type & Status', # Обновляем заголовок легенды
        height=700,
        width=1200
    )

    fig.show()

data = df.copy()

# Список city_id, которые нужно выделить
highlight_city_ids = [4515, 4180] 
v1_candidates = [4271, 4236, 5504, 5548, 4244]
v2_running = [4373]
v2_candidates = [4349, 9415, 4197, 4376, 5698, 4200, 4198, 4404, 5473, 5640, 5681]
data['highlight_group'] = data.apply(lambda row: 'v2_running' if (row['city_id'] in v2_running and row['order_type'] == 'moto_econom')
                                     else ('v2_candidates' if row['city_id'] in v2_candidates 
                                           else ('v1_candidates' if row['city_id'] in v1_candidates 
                                                 else ('v1_exp_done' if row['city_id'] in highlight_city_ids else 'Normal'))), axis=1)
cluster_plot(data)

### Alpha

In [3]:
start_date = '2025-01-29'
stop_date = '2025-02-26'
MIN_COMPLETED_ORDERS = 300000

In [ ]:
df_bids = bids_data(start_date=start_date, stop_date=stop_date, printBool=True)
df_bids.to_csv(f'data/ChooseCities_{start_date}_{stop_date}.csv', index=False)

In [4]:
df_bids = pd.read_csv(f'data/ChooseCities_{start_date}_{stop_date}.csv')

In [5]:
def get_color_for_alpha(alpha_str, cmap):
    """
    Находит цвет в словаре color_map по первой части ключа (до запятой).
    Предполагается, что для одного и того же alpha_str все соответствующие
    значения цвета будут одинаковыми (как в вашем примере).
    """
    # Формируем префикс, который должен быть в начале ключа
    # Добавляем запятую, чтобы '0.1' не совпало с '0.15'
    prefix_to_match = alpha_str + ","
    for key, value in cmap.items():
        if key.startswith(prefix_to_match):
            return value # Возвращаем значение первого найденного совпадения
    return None # Если совпадений не найдено

# Получаем цвет
# found_color = get_color_for_alpha(search_alpha_value, color_map)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import statsmodels.api as sm


# Список city_id для выделения
highlight_city_ids = [4515, 4180] 
v1_candidates = [4271, 4236, 5504, 5548, 4244]
v2_running = [4373]
v2_candidates = [4349, 9415, 4197, 4376, 5698, 4200, 4198, 4404]

df_filtered = df_bids.copy()
df_filtered['alpha_value_str'] = df_filtered['alpha_value'].astype(str)

df_filtered['highlight_status'] = df_filtered.apply(
    lambda row: 'v2_running' if (row['city_id'] in v2_running and row['order_type'] == 'moto_econom')
    else ('v2_candidates' if row['city_id'] in v2_candidates 
          else ('v1_candidates' if row['city_id'] in v1_candidates 
                else ('v1_exp_done' if row['city_id'] in highlight_city_ids else 'Normal'))), axis=1)

df_filtered = df_filtered[df_filtered['rides_count'] >= MIN_COMPLETED_ORDERS].copy()

if df_filtered.empty:
    print(f"DataFrame пуст после фильтрации с MIN_COMPLETED_ORDERS = {MIN_COMPLETED_ORDERS}. График не будет построен.")
else:
    fig = px.scatter(df_filtered,
                     x='bad_bids_share',
                     y='rides_badbids_share',
                     color='alpha_value_str',
                     symbol='highlight_status',
                    #  symbol_map={
                    #      'Highlighted City': 'star',
                    #      'Normal City': 'circle'
                    #  },
                     hover_data=df_filtered.columns.tolist(),
                     title='Impact of Alpha (Color) & City Highlight (Symbol) with OLS through Origin',
                     labels={
                         'share_of_bad_bids': 'Share of Bad Bids',
                         'share_done_orders_with_bad_tender': 'Share of Completed Orders with Bad Tender',
                         'alpha_value_str': 'Alpha Value',
                         'highlight_status': 'City Status'
                     }
                    )

    alpha_color_map = {}
    for trace in fig.data:
        alpha_val_for_trace = trace.legendgroup
        if alpha_val_for_trace not in alpha_color_map:
            alpha_color_map[alpha_val_for_trace] = trace.marker.color
    print(alpha_color_map)

    unique_alpha_values = sorted(df_filtered['alpha_value_str'].unique())

    for current_alpha_str in unique_alpha_values:
        # Данные для OLS берутся по ВСЕМ точкам для текущего current_alpha_str
        group_data_for_ols = df_filtered[df_filtered['alpha_value_str'] == current_alpha_str]

        x_data = group_data_for_ols['bad_bids_share']
        y_data = group_data_for_ols['rides_badbids_share']

        valid_indices = x_data.notna() & y_data.notna()
        x_clean = x_data[valid_indices]
        y_clean = y_data[valid_indices]

        if len(x_clean) >= 2:
            try:
                model = sm.OLS(y_clean, x_clean)
                results = model.fit()

                if results.params.empty:
                    print(f"Предупреждение: Параметры OLS пусты для alpha_value_str = {current_alpha_str}.")
                    continue
                
                slope = results.params.iloc[0]

                if pd.notna(slope) and np.isfinite(slope):
                    x_range_for_line = np.array([-0.05, 1.0])
                    y_line = slope * x_range_for_line
                    
                    line_color = get_color_for_alpha(current_alpha_str, alpha_color_map)
                    if line_color is None:
                        print(f"Предупреждение: Цвет не найден для OLS alpha_value_str = {current_alpha_str}. Используется черный.")
                        line_color = 'rgba(0,0,0,0.5)'

                    fig.add_trace(go.Scatter(
                        x=x_range_for_line,
                        y=y_line,
                        mode='lines',
                        name=f'OLS Trend ({current_alpha_str})',
                        line=dict(color=line_color, width=2, dash='dash'),
                        legendgroup=f"group_{current_alpha_str}", 
                        showlegend=True
                    ))
                else:
                    print(f"Предупреждение: Наклон (slope) NaN/Inf для alpha_value_str = {current_alpha_str}.")
            except Exception as e:
                print(f"Ошибка OLS для alpha_value_str = {current_alpha_str}: {e}")
        else:
            print(f"Мало данных (нужно >= 2, есть {len(x_clean)}) для OLS alpha_value_str = {current_alpha_str}.")

    fig.update_layout(
        xaxis=dict(range=[-0.05, 1]),
        yaxis=dict(range=[-0.05, 1]),
        height=1000,
        legend_title_text="Legend"
    )

    fig.show()

In [ ]:
# Параметры для v1_exp_done
4180: 0.1 => 10% rides_badbids_share, 28% badbids_share
4515: 0.125 => 20% rides_badbids_share, 38% badbids_share

### V1 Candidates

#### Veracruz 4180

In [412]:
max_bad_bids_share = df_bids[(df_bids['city_id'] == 4180) & 
                             (df_bids['order_type'] == 'auto_econom') &
                             (df_bids['alpha_value'] == 0.1)]['bad_bids_share'].max()

max_rides_badbids_share = df_bids[(df_bids['city_id'] == 4180) & 
                                  (df_bids['order_type'] == 'auto_econom') &
                                  (df_bids['alpha_value'] == 0.1)]['rides_badbids_share'].max()

In [413]:
reference_points_data = df_bids[(df_bids['city_id'] == 4180) & 
                                (df_bids['order_type'] == 'auto_econom') &
                                (df_bids['alpha_value'] <= 0.2) & 
                                (df_bids['alpha_value'] >= 0.1)]

In [414]:
# Assume df_bids is a pre-existing DataFrame in your environment
# Список city_id для выделения
# highlight_city_ids = [4515, 4180]
# v1_candidates = [4271, 4236, 5504, 5548, 4244]
# v2_running = [4373]
# v2_candidates = [4349, 9415, 4197, 4376, 5698, 4200, 4198, 4404]

df_filtered = df_bids.copy()
df_filtered['alpha_value_str'] = df_filtered['alpha_value'].astype(str)

# df_filtered['highlight_status'] = df_filtered.apply(
#     lambda row: 'v2_running' if (row['city_id'] in v2_running and row['order_type'] == 'moto_econom')
#     else ('v2_candidates' if row['city_id'] in v2_candidates
#           else ('v1_candidates' if row['city_id'] in v1_candidates
#                 else ('v1_exp_done' if row['city_id'] in highlight_city_ids else 'Normal'))), axis=1)

df_filtered = df_filtered[df_filtered['rides_count'] >= MIN_COMPLETED_ORDERS].copy()

# This will be the requested table
result_df = pd.DataFrame()
ols_slopes_for_table = {}

if df_filtered.empty:
    print(f"DataFrame пуст после фильтрации с MIN_COMPLETED_ORDERS = {MIN_COMPLETED_ORDERS}. График и таблица не будут построены.")
else:
    unique_alpha_values = sorted(df_filtered['alpha_value_str'].unique())

    for current_alpha_str in unique_alpha_values:
        group_data_for_ols = df_filtered[df_filtered['alpha_value_str'] == current_alpha_str]

        x_data = group_data_for_ols['bad_bids_share']
        y_data = group_data_for_ols['rides_badbids_share']

        valid_indices = x_data.notna() & y_data.notna()
        x_clean = x_data[valid_indices]
        y_clean = y_data[valid_indices]

        current_slope = np.nan # Default to NaN

        if len(x_clean) >= 2:
            try:
                model = sm.OLS(y_clean, x_clean) # OLS through origin
                results = model.fit()

                if results.params.empty:
                    print(f"Предупреждение: Параметры OLS пусты для alpha_value_str = {current_alpha_str}.")
                else:
                    slope_val = results.params.iloc[0]
                    if pd.notna(slope_val) and np.isfinite(slope_val):
                        current_slope = slope_val
                        x_range_for_line = np.array([-0.05, 1.0])
                        y_line = current_slope * x_range_for_line
                    else:
                        print(f"Предупреждение: Наклон (slope) NaN/Inf для alpha_value_str = {current_alpha_str}.")
            except Exception as e:
                print(f"Ошибка OLS для alpha_value_str = {current_alpha_str}: {e}")
        else:
            print(f"Мало данных (нужно >= 2, есть {len(x_clean)}) для OLS alpha_value_str = {current_alpha_str}.")
        
        ols_slopes_for_table[current_alpha_str] = current_slope

    # --- Start: PandaS TABLE GENERATION ---
    result_df = df_filtered.copy()

    # 2. Calculate OLS predicted value for each point
    def calculate_ols_predicted_value(row, reference_points_data=reference_points_data):
        alpha_str = row['alpha_value_str']
        bad_bids_val = row['bad_bids_share']
        slope = ols_slopes_for_table.get(alpha_str, np.nan) # Get slope, default to NaN
        
        if pd.notna(slope) and pd.notna(bad_bids_val):
            return slope * bad_bids_val
        return np.nan

    result_df['ols_predicted_value'] = result_df.apply(calculate_ols_predicted_value, axis=1)
    result_df['ols_diff'] = result_df['ols_predicted_value'] - result_df['rides_badbids_share']
    
    # Handle division by zero explicitly
    numerator = abs(result_df['ols_predicted_value'] - result_df['rides_badbids_share'])
    denominator = result_df['ols_predicted_value']

    # Where denominator is 0, result will be NaN. If numerator is also 0, it's NaN.
    # If numerator is non-zero and denominator is 0, it's inf.
    # To avoid inf and set to NaN instead, or a large number, or 0, you can use np.where:
    result_df['rel_diff'] = np.where(
        denominator != 0,  # Condition
        numerator / denominator,  # Value if true
        np.nan  # Value if false (e.g., set to NaN if denominator is 0)
        # You could also put 0 or a very large number here depending on how you want to treat it
    )

    # If you want to replace inf values that might have been created by the original formula:
    # result_df['rel_diff'].replace([np.inf, -np.inf], np.nan, inplace=True)    

    # 3. Calculate minimum distance to specified highlighted points
    # These are points in df_filtered where city_id is in highlight_city_ids
    # reference_points_data = df_filtered[df_filtered['city_id'].isin(highlight_city_ids)][['bad_bids_share', 'rides_badbids_share']].copy()
    # reference_points_data.dropna(inplace=True) 

    reference_coords_list = []
    if not reference_points_data.empty:
        reference_coords_list = list(zip(reference_points_data['bad_bids_share'], reference_points_data['rides_badbids_share']))

    def calculate_min_distance_to_ref_points(row, ref_coords):
        p1_x, p1_y = row['bad_bids_share'], row['rides_badbids_share']

        if pd.isna(p1_x) or pd.isna(p1_y) or not ref_coords:
            return np.nan

        min_dist_sq = np.inf # Using squared distance to avoid sqrt until the end
        
        for p2_x, p2_y in ref_coords:
            # Ensure reference point coordinates are valid
            if pd.notna(p2_x) and pd.notna(p2_y):
                dist_sq = (p1_x - p2_x)**2 + (p1_y - p2_y)**2
                if dist_sq < min_dist_sq:
                    min_dist_sq = dist_sq
        
        return np.sqrt(min_dist_sq) if min_dist_sq != np.inf else np.nan

    if not reference_coords_list:
        print("Предупреждение: Нет референсных точек (city_id in highlight_city_ids с валидными координатами) для расчета расстояний. Колонка 'min_distance_to_highlight' будет заполнена NaN.")
        result_df['min_distance_to_highlight'] = np.nan
    else:
        result_df['min_distance_to_highlight'] = result_df.apply(
            lambda row: calculate_min_distance_to_ref_points(row, reference_coords_list), axis=1
        )
    
    print("\nТаблица 'result_df' успешно создана.")


Таблица 'result_df' успешно создана.


In [415]:
# # Список city_id для выделения
# veracrus_candidates = result_df[#(result_df['city_id'] == 4180) & 
#           (result_df['order_type'] == 'auto_econom') &
#           (result_df['bad_bids_share'] >= 0.1) &
#           # (result_df['bad_bids_share'] <= max_bad_bids_share) &
#           (result_df['rides_badbids_share'] <= max_rides_badbids_share) &
#           (result_df['ols_diff'] >= 0.0) &
#           (result_df['rel_diff'] >= 0.25) &
#           (result_df['rel_diff'] <= 0.4) &
#           (result_df['alpha_value'] >= 0.15)
#           # & (result_df['min_distance_to_highlight'] < 0.05)
#           ]

In [416]:
df_filtered = result_df.copy()
df_filtered['alpha_value_str'] = df_filtered['alpha_value'].astype(str)

df_filtered['highlight_status'] = 'Other'
df_filtered.loc[veracrus_candidates.index, 'highlight_status'] = 'veracrus_candidates'
try:
    df_filtered.loc[veracrus_candidates_refined.index, 'highlight_status'] = 'veracrus_candidates_refined'
except:
    pass
df_filtered.loc[reference_points_data.index, 'highlight_status'] = 'veracrus'

city_ids_for_veracrus = [4271, 4515, 4236, 5504, 5548]
df_filtered.loc[df_filtered['city_id'].isin(city_ids_for_veracrus), 'highlight_status'] = 'veracrus_cluster'

df_filtered = df_filtered[df_filtered['rides_count'] >= MIN_COMPLETED_ORDERS].copy()

if df_filtered.empty:
    print(f"DataFrame пуст после фильтрации с MIN_COMPLETED_ORDERS = {MIN_COMPLETED_ORDERS}. График не будет построен.")
else:
    fig = px.scatter(df_filtered,
                     x='bad_bids_share',
                     y='rides_badbids_share',
                     color='alpha_value_str',
                     symbol='highlight_status',
                     symbol_map={
                         'veracrus': 'star',
                         'veracrus_candidates': 'cross'
                     },
                     hover_data=df_filtered.columns.tolist(),
                     title='Impact of Alpha (Color) & City Highlight (Symbol) with OLS through Origin',
                     labels={
                         'share_of_bad_bids': 'Share of Bad Bids',
                         'share_done_orders_with_bad_tender': 'Share of Completed Orders with Bad Tender',
                         'alpha_value_str': 'Alpha Value',
                         'highlight_status': 'City Status'
                     }
                    )

    alpha_color_map = {}
    for trace in fig.data:
        alpha_val_for_trace = trace.legendgroup
        if alpha_val_for_trace not in alpha_color_map:
            alpha_color_map[alpha_val_for_trace] = trace.marker.color
    print(alpha_color_map)

    unique_alpha_values = sorted(df_filtered['alpha_value_str'].unique())

    for current_alpha_str in unique_alpha_values:
        # Данные для OLS берутся по ВСЕМ точкам для текущего current_alpha_str
        group_data_for_ols = df_filtered[df_filtered['alpha_value_str'] == current_alpha_str]

        x_data = group_data_for_ols['bad_bids_share']
        y_data = group_data_for_ols['rides_badbids_share']

        valid_indices = x_data.notna() & y_data.notna()
        x_clean = x_data[valid_indices]
        y_clean = y_data[valid_indices]

        if len(x_clean) >= 2:
            try:
                model = sm.OLS(y_clean, x_clean)
                results = model.fit()

                if results.params.empty:
                    print(f"Предупреждение: Параметры OLS пусты для alpha_value_str = {current_alpha_str}.")
                    continue
                
                slope = results.params.iloc[0]

                if pd.notna(slope) and np.isfinite(slope):
                    x_range_for_line = np.array([-0.05, 1.0])
                    y_line = slope * x_range_for_line
                    
                    line_color = get_color_for_alpha(current_alpha_str, alpha_color_map)
                    if line_color is None:
                        print(f"Предупреждение: Цвет не найден для OLS alpha_value_str = {current_alpha_str}. Используется черный.")
                        line_color = 'rgba(0,0,0,0.5)'

                    fig.add_trace(go.Scatter(
                        x=x_range_for_line,
                        y=y_line,
                        mode='lines',
                        name=f'OLS Trend ({current_alpha_str})',
                        line=dict(color=line_color, width=2, dash='dash'),
                        legendgroup=f"group_{current_alpha_str}", 
                        showlegend=True
                    ))
                else:
                    print(f"Предупреждение: Наклон (slope) NaN/Inf для alpha_value_str = {current_alpha_str}.")
            except Exception as e:
                print(f"Ошибка OLS для alpha_value_str = {current_alpha_str}: {e}")
        else:
            print(f"Мало данных (нужно >= 2, есть {len(x_clean)}) для OLS alpha_value_str = {current_alpha_str}.")

    fig.update_layout(
        xaxis=dict(range=[-0.05, 1]),
        yaxis=dict(range=[-0.05, 1]),
        height=1000,
        legend_title_text="Legend"
    )

    fig.show()

{'0.1, Other': '#636efa', '0.1, veracrus': '#636efa', '0.1, veracrus_cluster': '#636efa', '0.125, Other': '#EF553B', '0.125, veracrus': '#EF553B', '0.125, veracrus_cluster': '#EF553B', '0.15, Other': '#00cc96', '0.15, veracrus_candidates': '#00cc96', '0.15, veracrus': '#00cc96', '0.15, veracrus_cluster': '#00cc96', '0.2, Other': '#ab63fa', '0.2, veracrus_candidates': '#ab63fa', '0.2, veracrus': '#ab63fa', '0.2, veracrus_cluster': '#ab63fa', '0.25, Other': '#FFA15A', '0.25, veracrus_candidates': '#FFA15A', '0.25, veracrus_cluster': '#FFA15A', '0.3, Other': '#19d3f3', '0.3, veracrus_candidates': '#19d3f3', '0.3, veracrus_cluster': '#19d3f3', '0.35, Other': '#FF6692', '0.35, veracrus_candidates': '#FF6692', '0.35, veracrus_cluster': '#FF6692', '0.4, Other': '#B6E880', '0.4, veracrus_candidates': '#B6E880', '0.4, veracrus_cluster': '#B6E880'}


In [304]:
# veracrus_candidates_refined = veracrus_candidates[(veracrus_candidates['bad_bids_share'] >= 0.15) &
#                                                   (veracrus_candidates['rides_badbids_share'] <= 0.07) &
#                                                   (veracrus_candidates['rides_badbids_share'] >= 0.05) &
#                                                   (veracrus_candidates['alpha_value'] >= 0.15)] #

# veracrus_candidates_refined = veracrus_candidates_refined[~veracrus_candidates_refined['city_id'].isin(reference_points_data['city_id'])]
# veracrus_candidates_refined

final_list = [
    {'city_id': 5504, 'alpha_value_numeric': 0.2},
    {'city_id': 5548, 'alpha_value_numeric': 0.2},
    {'city_id': 4271, 'alpha_value_numeric': 0.15}
]

final_list_df = pd.DataFrame(final_list)
final_list_df['alpha_value_str'] = final_list_df['alpha_value_numeric'].astype(str)
filter_final_list_df_for_merge = final_list_df[['city_id', 'alpha_value_str']]
selected_rows_df_veracrus = pd.merge(df_filtered, filter_final_list_df_for_merge, on=['city_id', 'alpha_value_str'], how='inner')
selected_rows_df_veracrus

,city_id,order_type,avg_rta,alpha_value,bids_count,bad_bids_count,bad_bids_share,rides_count,rides_badbids_count,rides_badbids_share,alpha_value_str,ols_predicted_value,ols_diff,rel_diff,min_distance_to_highlight,highlight_status
0,4271,auto_econom,200.416556,0.15,1901412.0,253299.0,0.133216,594691.0,26782.0,0.045035,0.15,0.061871,0.016835,0.272106,0.039107,veracrus_cluster
1,5504,auto_econom,208.292639,0.20,2315121.0,362965.0,0.156780,671074.0,34667.0,0.051659,0.2,0.061387,0.009728,0.158464,0.047049,veracrus_cluster
2,5548,auto_econom,270.392303,0.20,4609340.0,692657.0,0.150272,1348288.0,51635.0,0.038297,0.2,0.058838,0.020542,0.349121,0.052559,veracrus_cluster


In [305]:
# df_filtered[df_filtered['highlight_status'] == 'veracrus_candidates_refined'].sort_values('rides_count', ascending=False)

In [417]:
def cluster_plot(df):
    # 2. Используем новую колонку для параметра 'symbol'
    fig = px.scatter(
        df,
        x='order2bid',
        y='bid2done',
        # size='total_orders',
        color='order_type',
        symbol='highlight_group',  # Используем новую колонку для символов
        symbol_map={ # Опционально: задаем конкретные символы
            'Highlighted': 'star',
            'Normal': 'circle'
        },
        hover_name='city_id',
        hover_data={
            'city_id': True,
            'order_type': True,
            'total_orders': ':,',
            'total_rides': ':,',
            'bid2done': ':.4f',
            'order2bid': ':.4f',
            'highlight_group': True # Можно добавить в hover_data для ясности
        },
        title=f'Order2Bid (MR) vs Bid2Done (with Highlighted Cities), {start_date} - {stop_date}',
        labels={
            'bid2done': 'Bid2Done',
            'order2bid': 'Order2Bid',
            'total_orders': 'Total Orders',
            'total_rides': 'Total Rides',
            'order_type': 'Order Type',
            'city_id': 'City ID',
            'highlight_group': 'Highlight Status' # Лейбл для новой легенды символов
        }
    )

    fig.update_layout(
        legend_title_text='Order Type & Status', # Обновляем заголовок легенды
        height=700,
        width=1200
    )

    fig.show()

data = df.copy()

# Список city_id, которые нужно выделить
highlight_city_ids = [4180] 
# v1_candidates = veracrus_candidates_refined['city_id'].tolist()
data['highlight_group'] = data.apply(lambda row: 'veracruz' if row['city_id'] in highlight_city_ids 
                                     else ('v1_candidates' if row['city_id'] in v1_candidates else 'Normal'), axis=1)
cluster_plot(data)

In [270]:
city_ids_for_veracrus = [4271, 4515, 4236, 5504, 5548]

#### Puerto Vallarta 4515

In [315]:
max_bad_bids_share = df_bids[(df_bids['city_id'] == 4515) & 
                             (df_bids['order_type'] == 'auto_econom') &
                             (df_bids['alpha_value'] == 0.125)]['bad_bids_share'].max()

max_rides_badbids_share = df_bids[(df_bids['city_id'] == 4515) & 
                                  (df_bids['order_type'] == 'auto_econom') &
                                  (df_bids['alpha_value'] == 0.125)]['rides_badbids_share'].max()

In [316]:
reference_points_data = df_bids[(df_bids['city_id'] == 4515) & 
                                (df_bids['order_type'] == 'auto_econom') &
                                (df_bids['alpha_value'] <= 0.3) & 
                                (df_bids['alpha_value'] >= 0.125)]

In [317]:
# Assume df_bids is a pre-existing DataFrame in your environment
# Список city_id для выделения
# highlight_city_ids = [4515, 4180]
# v1_candidates = [4271, 4236, 5504, 5548, 4244]
# v2_running = [4373]
# v2_candidates = [4349, 9415, 4197, 4376, 5698, 4200, 4198, 4404]

df_filtered = df_bids.copy()
df_filtered['alpha_value_str'] = df_filtered['alpha_value'].astype(str)

# df_filtered['highlight_status'] = df_filtered.apply(
#     lambda row: 'v2_running' if (row['city_id'] in v2_running and row['order_type'] == 'moto_econom')
#     else ('v2_candidates' if row['city_id'] in v2_candidates
#           else ('v1_candidates' if row['city_id'] in v1_candidates
#                 else ('v1_exp_done' if row['city_id'] in highlight_city_ids else 'Normal'))), axis=1)

df_filtered = df_filtered[df_filtered['rides_count'] >= MIN_COMPLETED_ORDERS].copy()

# This will be the requested table
result_df = pd.DataFrame()
ols_slopes_for_table = {}

if df_filtered.empty:
    print(f"DataFrame пуст после фильтрации с MIN_COMPLETED_ORDERS = {MIN_COMPLETED_ORDERS}. График и таблица не будут построены.")
else:
    unique_alpha_values = sorted(df_filtered['alpha_value_str'].unique())

    for current_alpha_str in unique_alpha_values:
        group_data_for_ols = df_filtered[df_filtered['alpha_value_str'] == current_alpha_str]

        x_data = group_data_for_ols['bad_bids_share']
        y_data = group_data_for_ols['rides_badbids_share']

        valid_indices = x_data.notna() & y_data.notna()
        x_clean = x_data[valid_indices]
        y_clean = y_data[valid_indices]

        current_slope = np.nan # Default to NaN

        if len(x_clean) >= 2:
            try:
                model = sm.OLS(y_clean, x_clean) # OLS through origin
                results = model.fit()

                if results.params.empty:
                    print(f"Предупреждение: Параметры OLS пусты для alpha_value_str = {current_alpha_str}.")
                else:
                    slope_val = results.params.iloc[0]
                    if pd.notna(slope_val) and np.isfinite(slope_val):
                        current_slope = slope_val
                        x_range_for_line = np.array([-0.05, 1.0])
                        y_line = current_slope * x_range_for_line
                    else:
                        print(f"Предупреждение: Наклон (slope) NaN/Inf для alpha_value_str = {current_alpha_str}.")
            except Exception as e:
                print(f"Ошибка OLS для alpha_value_str = {current_alpha_str}: {e}")
        else:
            print(f"Мало данных (нужно >= 2, есть {len(x_clean)}) для OLS alpha_value_str = {current_alpha_str}.")
        
        ols_slopes_for_table[current_alpha_str] = current_slope

    # --- Start: PandaS TABLE GENERATION ---
    result_df = df_filtered.copy()

    # 2. Calculate OLS predicted value for each point
    def calculate_ols_predicted_value(row, reference_points_data=reference_points_data):
        alpha_str = row['alpha_value_str']
        bad_bids_val = row['bad_bids_share']
        slope = ols_slopes_for_table.get(alpha_str, np.nan) # Get slope, default to NaN
        
        if pd.notna(slope) and pd.notna(bad_bids_val):
            return slope * bad_bids_val
        return np.nan

    result_df['ols_predicted_value'] = result_df.apply(calculate_ols_predicted_value, axis=1)
    result_df['ols_diff'] = result_df['ols_predicted_value'] - result_df['rides_badbids_share']
    
    # Handle division by zero explicitly
    numerator = abs(result_df['ols_predicted_value'] - result_df['rides_badbids_share'])
    denominator = result_df['ols_predicted_value']

    # Where denominator is 0, result will be NaN. If numerator is also 0, it's NaN.
    # If numerator is non-zero and denominator is 0, it's inf.
    # To avoid inf and set to NaN instead, or a large number, or 0, you can use np.where:
    result_df['rel_diff'] = np.where(
        denominator != 0,  # Condition
        numerator / denominator,  # Value if true
        np.nan  # Value if false (e.g., set to NaN if denominator is 0)
        # You could also put 0 or a very large number here depending on how you want to treat it
    )

    # If you want to replace inf values that might have been created by the original formula:
    # result_df['rel_diff'].replace([np.inf, -np.inf], np.nan, inplace=True)    

    # 3. Calculate minimum distance to specified highlighted points
    # These are points in df_filtered where city_id is in highlight_city_ids
    # reference_points_data = df_filtered[df_filtered['city_id'].isin(highlight_city_ids)][['bad_bids_share', 'rides_badbids_share']].copy()
    # reference_points_data.dropna(inplace=True) 

    reference_coords_list = []
    if not reference_points_data.empty:
        reference_coords_list = list(zip(reference_points_data['bad_bids_share'], reference_points_data['rides_badbids_share']))

    def calculate_min_distance_to_ref_points(row, ref_coords):
        p1_x, p1_y = row['bad_bids_share'], row['rides_badbids_share']

        if pd.isna(p1_x) or pd.isna(p1_y) or not ref_coords:
            return np.nan

        min_dist_sq = np.inf # Using squared distance to avoid sqrt until the end
        
        for p2_x, p2_y in ref_coords:
            # Ensure reference point coordinates are valid
            if pd.notna(p2_x) and pd.notna(p2_y):
                dist_sq = (p1_x - p2_x)**2 + (p1_y - p2_y)**2
                if dist_sq < min_dist_sq:
                    min_dist_sq = dist_sq
        
        return np.sqrt(min_dist_sq) if min_dist_sq != np.inf else np.nan

    if not reference_coords_list:
        print("Предупреждение: Нет референсных точек (city_id in highlight_city_ids с валидными координатами) для расчета расстояний. Колонка 'min_distance_to_highlight' будет заполнена NaN.")
        result_df['min_distance_to_highlight'] = np.nan
    else:
        result_df['min_distance_to_highlight'] = result_df.apply(
            lambda row: calculate_min_distance_to_ref_points(row, reference_coords_list), axis=1
        )
    
    print("\nТаблица 'result_df' успешно создана.")


Таблица 'result_df' успешно создана.


In [318]:
# # Список city_id для выделения
# puerto_candidates = result_df[#(result_df['city_id'] == 4515) & 
#           (result_df['order_type'] == 'auto_econom') &
#           (result_df['bad_bids_share'] >= 0.1) &
#         #   (result_df['bad_bids_share'] <= max_bad_bids_share) &
#           (result_df['rides_badbids_share'] <= max_rides_badbids_share) &
#           (result_df['ols_diff'] <= 0.0) &
#           (result_df['rel_diff'] >= 0.05) &
#           (result_df['rel_diff'] <= 0.25) &
#           (result_df['alpha_value'] >= 0.15)
#         #   (result_df['min_distance_to_highlight'] < 0.07)
#           ]

In [319]:
df_filtered = result_df.copy()
df_filtered['alpha_value_str'] = df_filtered['alpha_value'].astype(str)

df_filtered['highlight_status'] = 'Other'
df_filtered.loc[puerto_candidates.index, 'highlight_status'] = 'puerto_candidates'
try:
    df_filtered.loc[puerto_candidates_refined.index, 'highlight_status'] = 'puerto_candidates_refined'
except:
    pass
df_filtered.loc[reference_points_data.index, 'highlight_status'] = 'puerto'

city_ids_for_puerto = [4180, 4271, 4236, 5504, 5548, 4240, 4258, 5512, 262, 228, 4227]
df_filtered.loc[df_filtered['city_id'].isin(city_ids_for_puerto), 'highlight_status'] = 'puerto_cluster'

df_filtered = df_filtered[df_filtered['rides_count'] >= MIN_COMPLETED_ORDERS].copy()

if df_filtered.empty:
    print(f"DataFrame пуст после фильтрации с MIN_COMPLETED_ORDERS = {MIN_COMPLETED_ORDERS}. График не будет построен.")
else:
    fig = px.scatter(df_filtered,
                     x='bad_bids_share',
                     y='rides_badbids_share',
                     color='alpha_value_str',
                     symbol='highlight_status',
                     symbol_map={
                         'puerto': 'star',
                         'puerto_candidates': 'cross'
                     },
                     hover_data=df_filtered.columns.tolist(),
                     title='Impact of Alpha (Color) & City Highlight (Symbol) with OLS through Origin',
                     labels={
                         'share_of_bad_bids': 'Share of Bad Bids',
                         'share_done_orders_with_bad_tender': 'Share of Completed Orders with Bad Tender',
                         'alpha_value_str': 'Alpha Value',
                         'highlight_status': 'City Status'
                     }
                    )

    alpha_color_map = {}
    for trace in fig.data:
        alpha_val_for_trace = trace.legendgroup
        if alpha_val_for_trace not in alpha_color_map:
            alpha_color_map[alpha_val_for_trace] = trace.marker.color
    print(alpha_color_map)

    unique_alpha_values = sorted(df_filtered['alpha_value_str'].unique())

    for current_alpha_str in unique_alpha_values:
        # Данные для OLS берутся по ВСЕМ точкам для текущего current_alpha_str
        group_data_for_ols = df_filtered[df_filtered['alpha_value_str'] == current_alpha_str]

        x_data = group_data_for_ols['bad_bids_share']
        y_data = group_data_for_ols['rides_badbids_share']

        valid_indices = x_data.notna() & y_data.notna()
        x_clean = x_data[valid_indices]
        y_clean = y_data[valid_indices]

        if len(x_clean) >= 2:
            try:
                model = sm.OLS(y_clean, x_clean)
                results = model.fit()

                if results.params.empty:
                    print(f"Предупреждение: Параметры OLS пусты для alpha_value_str = {current_alpha_str}.")
                    continue
                
                slope = results.params.iloc[0]

                if pd.notna(slope) and np.isfinite(slope):
                    x_range_for_line = np.array([-0.05, 1.0])
                    y_line = slope * x_range_for_line
                    
                    line_color = get_color_for_alpha(current_alpha_str, alpha_color_map)
                    if line_color is None:
                        print(f"Предупреждение: Цвет не найден для OLS alpha_value_str = {current_alpha_str}. Используется черный.")
                        line_color = 'rgba(0,0,0,0.5)'

                    fig.add_trace(go.Scatter(
                        x=x_range_for_line,
                        y=y_line,
                        mode='lines',
                        name=f'OLS Trend ({current_alpha_str})',
                        line=dict(color=line_color, width=2, dash='dash'),
                        legendgroup=f"group_{current_alpha_str}", 
                        showlegend=True
                    ))
                else:
                    print(f"Предупреждение: Наклон (slope) NaN/Inf для alpha_value_str = {current_alpha_str}.")
            except Exception as e:
                print(f"Ошибка OLS для alpha_value_str = {current_alpha_str}: {e}")
        else:
            print(f"Мало данных (нужно >= 2, есть {len(x_clean)}) для OLS alpha_value_str = {current_alpha_str}.")

    fig.update_layout(
        xaxis=dict(range=[-0.05, 1]),
        yaxis=dict(range=[-0.05, 1]),
        height=1000,
        legend_title_text="Legend"
    )

    fig.show()

{'0.1, Other': '#636efa', '0.1, puerto_cluster': '#636efa', '0.125, Other': '#EF553B', '0.125, puerto_cluster': '#EF553B', '0.125, puerto': '#EF553B', '0.15, Other': '#00cc96', '0.15, puerto_candidates': '#00cc96', '0.15, puerto_cluster': '#00cc96', '0.15, puerto': '#00cc96', '0.2, Other': '#ab63fa', '0.2, puerto_candidates': '#ab63fa', '0.2, puerto_cluster': '#ab63fa', '0.2, puerto': '#ab63fa', '0.25, Other': '#FFA15A', '0.25, puerto_candidates': '#FFA15A', '0.25, puerto_cluster': '#FFA15A', '0.25, puerto': '#FFA15A', '0.3, Other': '#19d3f3', '0.3, puerto_candidates': '#19d3f3', '0.3, puerto_cluster': '#19d3f3', '0.3, puerto': '#19d3f3', '0.35, Other': '#FF6692', '0.35, puerto_candidates': '#FF6692', '0.35, puerto_cluster': '#FF6692', '0.4, Other': '#B6E880', '0.4, puerto_cluster': '#B6E880'}


In [320]:
# puerto_candidates_refined = puerto_candidates[(puerto_candidates['bad_bids_share'] >= 0.19) &
#                                                   (puerto_candidates['rides_badbids_share'] >= 0.09) &
#                                                   (puerto_candidates['rides_badbids_share'] <= 0.16) &
#                                                   (puerto_candidates['alpha_value'] >= 0.15)] #
# puerto_candidates_refined


final_list = [
    {'city_id': 262, 'alpha_value_numeric': 0.15},
    {'city_id': 4240, 'alpha_value_numeric': 0.15},
    {'city_id': 5504, 'alpha_value_numeric': 0.15},
    {'city_id': 4258, 'alpha_value_numeric': 0.20},
    {'city_id': 228, 'alpha_value_numeric': 0.20},
    {'city_id': 262, 'alpha_value_numeric': 0.20},
    {'city_id': 4227, 'alpha_value_numeric': 0.20},
]

final_list_df = pd.DataFrame(final_list)
final_list_df['alpha_value_str'] = final_list_df['alpha_value_numeric'].astype(str)
filter_final_list_df_for_merge = final_list_df[['city_id', 'alpha_value_str']]
selected_rows_df_puerto = pd.merge(df_filtered, filter_final_list_df_for_merge, on=['city_id', 'alpha_value_str'], how='inner')
selected_rows_df_puerto

,city_id,order_type,avg_rta,alpha_value,bids_count,bad_bids_count,bad_bids_share,rides_count,rides_badbids_count,rides_badbids_share,alpha_value_str,ols_predicted_value,ols_diff,rel_diff,min_distance_to_highlight,highlight_status
0,228,auto_econom,201.728708,0.20,2106598.0,389680.0,0.184981,511413.0,39849.0,0.077919,0.2,0.072428,-0.005491,0.075814,0.038609,puerto_cluster
1,262,auto_econom,208.435669,0.15,1216634.0,303646.0,0.249579,314198.0,34024.0,0.108288,0.15,0.115914,0.007625,0.065783,0.033580,puerto_cluster
2,262,auto_econom,208.435669,0.20,1216634.0,191336.0,0.157267,314198.0,18449.0,0.058718,0.2,0.061577,0.002859,0.046434,0.042429,puerto_cluster
3,4227,auto_econom,311.244127,0.20,3771375.0,541978.0,0.143708,974039.0,64501.0,0.066220,0.2,0.056268,-0.009952,0.176864,0.032209,puerto_cluster
4,4240,auto_econom,251.085345,0.15,1754872.0,440479.0,0.251003,455620.0,46709.0,0.102517,0.15,0.116575,0.014058,0.120590,0.033886,puerto_cluster
5,4258,auto_econom,171.632032,0.20,6095409.0,1849051.0,0.303351,1519645.0,174811.0,0.115034,0.2,0.118776,0.003742,0.031502,0.062459,puerto_cluster
6,5504,auto_econom,208.292639,0.15,2315121.0,591289.0,0.255403,671074.0,64447.0,0.096036,0.15,0.118619,0.022583,0.190383,0.038226,puerto_cluster


In [321]:
# df_filtered[df_filtered['highlight_status'] == 'puerto_candidates_refined'].sort_values('rides_count', ascending=False)

In [322]:
def cluster_plot(df):
    # 2. Используем новую колонку для параметра 'symbol'
    fig = px.scatter(
        df,
        x='order2bid',
        y='bid2done',
        # size='total_orders',
        color='order_type',
        symbol='highlight_group',  # Используем новую колонку для символов
        symbol_map={ # Опционально: задаем конкретные символы
            'Highlighted': 'star',
            'Normal': 'circle'
        },
        hover_name='city_id',
        hover_data={
            'city_id': True,
            'order_type': True,
            'total_orders': ':,',
            'total_rides': ':,',
            'bid2done': ':.4f',
            'order2bid': ':.4f',
            'highlight_group': True # Можно добавить в hover_data для ясности
        },
        title=f'Order2Bid (MR) vs Bid2Done (with Highlighted Cities), {start_date} - {stop_date}',
        labels={
            'bid2done': 'Bid2Done',
            'order2bid': 'Order2Bid',
            'total_orders': 'Total Orders',
            'total_rides': 'Total Rides',
            'order_type': 'Order Type',
            'city_id': 'City ID',
            'highlight_group': 'Highlight Status' # Лейбл для новой легенды символов
        }
    )

    fig.update_layout(
        legend_title_text='Order Type & Status', # Обновляем заголовок легенды
        height=700,
        width=1200
    )

    fig.show()

data = df.copy()

# Список city_id, которые нужно выделить
highlight_city_ids = [4515] 
v1_candidates = puerto_candidates_refined['city_id'].tolist()
data['highlight_group'] = data.apply(lambda row: 'puerto' if row['city_id'] in highlight_city_ids 
                                     else ('v1_candidates' if row['city_id'] in v1_candidates else 'Normal'), axis=1)
cluster_plot(data)

NameError: name 'puerto_candidates_refined' is not defined

In [276]:
del puerto_candidates_refined

NameError: name 'puerto_candidates_refined' is not defined

### V2 Candidates

#### Recife 4373

In [356]:
def cluster_plot(df):
    # 2. Используем новую колонку для параметра 'symbol'
    fig = px.scatter(
        df,
        x='order2bid',
        y='bid2done',
        # size='total_orders',
        color='order_type',
        symbol='highlight_group',  # Используем новую колонку для символов
        symbol_map={ # Опционально: задаем конкретные символы
            'Highlighted': 'star',
            'Normal': 'circle'
        },
        hover_name='city_id',
        hover_data={
            'city_id': True,
            'order_type': True,
            'total_orders': ':,',
            'total_rides': ':,',
            'bid2done': ':.4f',
            'order2bid': ':.4f',
            'highlight_group': True # Можно добавить в hover_data для ясности
        },
        title=f'Order2Bid (MR) vs Bid2Done (with Highlighted Cities), {start_date} - {stop_date}',
        labels={
            'bid2done': 'Bid2Done',
            'order2bid': 'Order2Bid',
            'total_orders': 'Total Orders',
            'total_rides': 'Total Rides',
            'order_type': 'Order Type',
            'city_id': 'City ID',
            'highlight_group': 'Highlight Status' # Лейбл для новой легенды символов
        }
    )

    fig.update_layout(
        legend_title_text='Order Type & Status', # Обновляем заголовок легенды
        height=700,
        width=1200
    )

    fig.show()

data = df.copy()

# Список city_id, которые нужно выделить
highlight_city_ids = [4373] 
v1_candidates = []
data['highlight_group'] = data.apply(lambda row: 'recife' if row['city_id'] in highlight_city_ids 
                                     else ('v1_candidates' if row['city_id'] in v1_candidates else 'Normal'), axis=1)
cluster_plot(data)

In [ ]:
def cluster_plot(df):
    # 2. Используем новую колонку для параметра 'symbol'
    fig = px.scatter(
        df,
        x='order2bid',
        y='bid2done',
        # size='total_orders',
        # color='order_type',
        color = 'highlight_group',
        # symbol='highlight_group',  # Используем новую колонку для символов
        symbol_map={ # Опционально: задаем конкретные символы
            'Highlighted': 'star',
            'Normal': 'circle'
        },
        hover_name='city_id',
        hover_data={
            'city': True,
            'country': True,
            'city_id': True,
            'order_type': True,
            'total_orders': ':,',
            'total_rides': ':,',
            'bid2done': ':.4f',
            'order2bid': ':.4f',
            'highlight_group': True # Можно добавить в hover_data для ясности
        },
        title=f'Order2Bid (MR) vs Bid2Done (with Highlighted Cities), {start_date} - {stop_date}',
        labels={
            'city': 'City',
            'country': 'Country',
            'bid2done': 'Bid2Done',
            'order2bid': 'Order2Bid',
            'total_orders': 'Total Orders',
            'total_rides': 'Total Rides',
            'order_type': 'Order Type',
            'city_id': 'City ID',
            'highlight_group': 'Highlight Status' # Лейбл для новой легенды символов
        }
    )

    fig.update_layout(
        legend_title_text='Order Type & Status', # Обновляем заголовок легенды
        height=800,
        width=1100
    )

    fig.show()

data = df.copy()
data = data[data['order_type'].isin(ok_types_list)]
data['order_type'] = 'Econom_My'

temp_df = df[df['city_id'] == 4373].copy()
temp_df['highlight_group'] = 'recife_moto'
data = pd.concat([data, temp_df], ignore_index=True)
data = pd.merge(names_df, data, on='city_id', how='right')

# Список city_id, которые нужно выделить
highlight_city_ids = [4373] 
v1_candidates = []
data['highlight_group'] = data.apply(lambda row: 'recife' if row['city_id'] in highlight_city_ids 
                                     else ('recife_cluster' if row['city_id'] in city_ids_for_recife else 
                                           ('rightlow_cluster' if row['city_id'] in rightlow_city_ids_for_v2 else
                                            ('middle_cluster' if row['city_id'] in middle_city_ids_for_v2 else'Normal'))), axis=1)
cluster_plot(data)

In [383]:
city_ids_for_recife =[4376, 4197]
rightlow_city_ids_for_v2= [5681, 5640, 5473, 5957, 4825, 5257, 4208, 5249]
middle_city_ids_for_v2 = [4267, 4271, 262, 228, 4231, 4194, 4243, 4234, 242, 235, 238, 150, 4404, 4198, 4242, 4229, 4300, 4252, 5513]

In [335]:
max_bad_bids_share = 1

max_rides_badbids_share = 1

In [336]:
df_filtered = df_bids.copy()
df_filtered['alpha_value_str'] = df_filtered['alpha_value'].astype(str)

# df_filtered['highlight_status'] = df_filtered.apply(
#     lambda row: 'v2_running' if (row['city_id'] in v2_running and row['order_type'] == 'moto_econom')
#     else ('v2_candidates' if row['city_id'] in v2_candidates
#           else ('v1_candidates' if row['city_id'] in v1_candidates
#                 else ('v1_exp_done' if row['city_id'] in highlight_city_ids else 'Normal'))), axis=1)

df_filtered = df_filtered[df_filtered['rides_count'] >= MIN_COMPLETED_ORDERS].copy()

# This will be the requested table
result_df = pd.DataFrame()
ols_slopes_for_table = {}

if df_filtered.empty:
    print(f"DataFrame пуст после фильтрации с MIN_COMPLETED_ORDERS = {MIN_COMPLETED_ORDERS}. График и таблица не будут построены.")
else:
    unique_alpha_values = sorted(df_filtered['alpha_value_str'].unique())

    for current_alpha_str in unique_alpha_values:
        group_data_for_ols = df_filtered[df_filtered['alpha_value_str'] == current_alpha_str]

        x_data = group_data_for_ols['bad_bids_share']
        y_data = group_data_for_ols['rides_badbids_share']

        valid_indices = x_data.notna() & y_data.notna()
        x_clean = x_data[valid_indices]
        y_clean = y_data[valid_indices]

        current_slope = np.nan # Default to NaN

        if len(x_clean) >= 2:
            try:
                model = sm.OLS(y_clean, x_clean) # OLS through origin
                results = model.fit()

                if results.params.empty:
                    print(f"Предупреждение: Параметры OLS пусты для alpha_value_str = {current_alpha_str}.")
                else:
                    slope_val = results.params.iloc[0]
                    if pd.notna(slope_val) and np.isfinite(slope_val):
                        current_slope = slope_val
                        x_range_for_line = np.array([-0.05, 1.0])
                        y_line = current_slope * x_range_for_line
                    else:
                        print(f"Предупреждение: Наклон (slope) NaN/Inf для alpha_value_str = {current_alpha_str}.")
            except Exception as e:
                print(f"Ошибка OLS для alpha_value_str = {current_alpha_str}: {e}")
        else:
            print(f"Мало данных (нужно >= 2, есть {len(x_clean)}) для OLS alpha_value_str = {current_alpha_str}.")
        
        ols_slopes_for_table[current_alpha_str] = current_slope

    # --- Start: PandaS TABLE GENERATION ---
    result_df = df_filtered.copy()

    # 2. Calculate OLS predicted value for each point
    def calculate_ols_predicted_value(row, reference_points_data=reference_points_data):
        alpha_str = row['alpha_value_str']
        bad_bids_val = row['bad_bids_share']
        slope = ols_slopes_for_table.get(alpha_str, np.nan) # Get slope, default to NaN
        
        if pd.notna(slope) and pd.notna(bad_bids_val):
            return slope * bad_bids_val
        return np.nan

    result_df['ols_predicted_value'] = result_df.apply(calculate_ols_predicted_value, axis=1)
    result_df['ols_diff'] = result_df['ols_predicted_value'] - result_df['rides_badbids_share']
    
    # Handle division by zero explicitly
    numerator = abs(result_df['ols_predicted_value'] - result_df['rides_badbids_share'])
    denominator = result_df['ols_predicted_value']

    # Where denominator is 0, result will be NaN. If numerator is also 0, it's NaN.
    # If numerator is non-zero and denominator is 0, it's inf.
    # To avoid inf and set to NaN instead, or a large number, or 0, you can use np.where:
    result_df['rel_diff'] = np.where(
        denominator != 0,  # Condition
        numerator / denominator,  # Value if true
        np.nan  # Value if false (e.g., set to NaN if denominator is 0)
        # You could also put 0 or a very large number here depending on how you want to treat it
    )

    # If you want to replace inf values that might have been created by the original formula:
    # result_df['rel_diff'].replace([np.inf, -np.inf], np.nan, inplace=True)    

    # 3. Calculate minimum distance to specified highlighted points
    # These are points in df_filtered where city_id is in highlight_city_ids
    # reference_points_data = df_filtered[df_filtered['city_id'].isin(highlight_city_ids)][['bad_bids_share', 'rides_badbids_share']].copy()
    # reference_points_data.dropna(inplace=True) 

    reference_coords_list = []
    if not reference_points_data.empty:
        reference_coords_list = list(zip(reference_points_data['bad_bids_share'], reference_points_data['rides_badbids_share']))

    def calculate_min_distance_to_ref_points(row, ref_coords):
        p1_x, p1_y = row['bad_bids_share'], row['rides_badbids_share']

        if pd.isna(p1_x) or pd.isna(p1_y) or not ref_coords:
            return np.nan

        min_dist_sq = np.inf # Using squared distance to avoid sqrt until the end
        
        for p2_x, p2_y in ref_coords:
            # Ensure reference point coordinates are valid
            if pd.notna(p2_x) and pd.notna(p2_y):
                dist_sq = (p1_x - p2_x)**2 + (p1_y - p2_y)**2
                if dist_sq < min_dist_sq:
                    min_dist_sq = dist_sq
        
        return np.sqrt(min_dist_sq) if min_dist_sq != np.inf else np.nan

    if not reference_coords_list:
        print("Предупреждение: Нет референсных точек (city_id in highlight_city_ids с валидными координатами) для расчета расстояний. Колонка 'min_distance_to_highlight' будет заполнена NaN.")
        result_df['min_distance_to_highlight'] = np.nan
    else:
        result_df['min_distance_to_highlight'] = result_df.apply(
            lambda row: calculate_min_distance_to_ref_points(row, reference_coords_list), axis=1
        )
    
    print("\nТаблица 'result_df' успешно создана.")


Таблица 'result_df' успешно создана.


In [410]:
df_filtered = result_df.copy()
df_filtered['alpha_value_str'] = df_filtered['alpha_value'].astype(str)

df_filtered['highlight_status'] = 'Other'

df_filtered.loc[df_filtered['city_id'].isin(city_ids_for_veracrus), 'highlight_status'] = 'recife_cluster'
df_filtered.loc[df_filtered['city_id'].isin(middle_city_ids_for_v2), 'highlight_status'] = 'middle_cluster'
df_filtered.loc[df_filtered['city_id'].isin(rightlow_city_ids_for_v2), 'highlight_status'] = 'rightlow_cluster'

try:
    df_filtered.loc[recife_cluster_refined.index, 'highlight_status'] = 'recife_cluster_refined'
    df_filtered.loc[middle_cluster_refined.index, 'highlight_status'] = 'middle_cluster_refined'
    df_filtered.loc[rightlow_cluster_refined.index, 'highlight_status'] = 'rightlow_cluster_refined'
except:
    pass

df_filtered = df_filtered[df_filtered['rides_count'] >= MIN_COMPLETED_ORDERS].copy()

if df_filtered.empty:
    print(f"DataFrame пуст после фильтрации с MIN_COMPLETED_ORDERS = {MIN_COMPLETED_ORDERS}. График не будет построен.")
else:
    fig = px.scatter(df_filtered,
                     x='bad_bids_share',
                     y='rides_badbids_share',
                     color='alpha_value_str',
                     symbol='highlight_status',
                     symbol_map={
                         'puerto': 'star',
                         'puerto_candidates': 'cross'
                     },
                     hover_data=df_filtered.columns.tolist(),
                     title='Impact of Alpha (Color) & City Highlight (Symbol) with OLS through Origin',
                     labels={
                         'share_of_bad_bids': 'Share of Bad Bids',
                         'share_done_orders_with_bad_tender': 'Share of Completed Orders with Bad Tender',
                         'alpha_value_str': 'Alpha Value',
                         'highlight_status': 'City Status'
                     }
                    )

    alpha_color_map = {}
    for trace in fig.data:
        alpha_val_for_trace = trace.legendgroup
        if alpha_val_for_trace not in alpha_color_map:
            alpha_color_map[alpha_val_for_trace] = trace.marker.color
    print(alpha_color_map)

    unique_alpha_values = sorted(df_filtered['alpha_value_str'].unique())

    for current_alpha_str in unique_alpha_values:
        # Данные для OLS берутся по ВСЕМ точкам для текущего current_alpha_str
        group_data_for_ols = df_filtered[df_filtered['alpha_value_str'] == current_alpha_str]

        x_data = group_data_for_ols['bad_bids_share']
        y_data = group_data_for_ols['rides_badbids_share']

        valid_indices = x_data.notna() & y_data.notna()
        x_clean = x_data[valid_indices]
        y_clean = y_data[valid_indices]

        if len(x_clean) >= 2:
            try:
                model = sm.OLS(y_clean, x_clean)
                results = model.fit()

                if results.params.empty:
                    print(f"Предупреждение: Параметры OLS пусты для alpha_value_str = {current_alpha_str}.")
                    continue
                
                slope = results.params.iloc[0]

                if pd.notna(slope) and np.isfinite(slope):
                    x_range_for_line = np.array([-0.05, 1.0])
                    y_line = slope * x_range_for_line
                    
                    line_color = get_color_for_alpha(current_alpha_str, alpha_color_map)
                    if line_color is None:
                        print(f"Предупреждение: Цвет не найден для OLS alpha_value_str = {current_alpha_str}. Используется черный.")
                        line_color = 'rgba(0,0,0,0.5)'

                    fig.add_trace(go.Scatter(
                        x=x_range_for_line,
                        y=y_line,
                        mode='lines',
                        name=f'OLS Trend ({current_alpha_str})',
                        line=dict(color=line_color, width=2, dash='dash'),
                        legendgroup=f"group_{current_alpha_str}", 
                        showlegend=True
                    ))
                else:
                    print(f"Предупреждение: Наклон (slope) NaN/Inf для alpha_value_str = {current_alpha_str}.")
            except Exception as e:
                print(f"Ошибка OLS для alpha_value_str = {current_alpha_str}: {e}")
        else:
            print(f"Мало данных (нужно >= 2, есть {len(x_clean)}) для OLS alpha_value_str = {current_alpha_str}.")

    fig.update_layout(
        xaxis=dict(range=[-0.05, 1]),
        yaxis=dict(range=[-0.05, 1]),
        height=1000,
        legend_title_text="Legend"
    )

    fig.show()

{'0.1, middle_cluster_refined': '#636efa', '0.1, middle_cluster': '#636efa', '0.1, Other': '#636efa', '0.1, rightlow_cluster_refined': '#636efa', '0.1, rightlow_cluster': '#636efa', '0.1, recife_cluster': '#636efa', '0.1, recife_cluster_refined': '#636efa', '0.125, middle_cluster_refined': '#EF553B', '0.125, middle_cluster': '#EF553B', '0.125, Other': '#EF553B', '0.125, rightlow_cluster_refined': '#EF553B', '0.125, rightlow_cluster': '#EF553B', '0.125, recife_cluster': '#EF553B', '0.125, recife_cluster_refined': '#EF553B', '0.15, middle_cluster_refined': '#00cc96', '0.15, middle_cluster': '#00cc96', '0.15, Other': '#00cc96', '0.15, rightlow_cluster_refined': '#00cc96', '0.15, rightlow_cluster': '#00cc96', '0.15, recife_cluster': '#00cc96', '0.15, recife_cluster_refined': '#00cc96', '0.2, middle_cluster_refined': '#ab63fa', '0.2, middle_cluster': '#ab63fa', '0.2, Other': '#ab63fa', '0.2, rightlow_cluster_refined': '#ab63fa', '0.2, rightlow_cluster': '#ab63fa', '0.2, recife_cluster': '#a

In [409]:
rightlow_cluster_refined

,city_id,order_type,avg_rta,alpha_value,bids_count,bad_bids_count,bad_bids_share,rides_count,rides_badbids_count,rides_badbids_share,alpha_value_str,ols_predicted_value,ols_diff,rel_diff,min_distance_to_highlight,highlight_status
3800,4208,auto_econom_dublicate,361.098065,0.100,3211709.0,753594.0,0.234640,666460.0,68028.0,0.102074,0.1,0.121988,0.019914,0.163248,0.017606,rightlow_cluster
3801,4208,auto_econom_dublicate,361.098065,0.125,3211709.0,645938.0,0.201120,666460.0,55103.0,0.082680,0.125,0.096350,0.013670,0.141880,0.023012,rightlow_cluster
8168,4825,auto_econom,316.736312,0.100,7917659.0,4761070.0,0.601323,1400624.0,394755.0,0.281842,0.1,0.312625,0.030783,0.098465,0.230261,rightlow_cluster
8169,4825,auto_econom,316.736312,0.125,7917659.0,4464157.0,0.563823,1400624.0,345584.0,0.246736,0.125,0.270110,0.023375,0.086537,0.183841,rightlow_cluster
8170,4825,auto_econom,316.736312,0.150,7917659.0,4208587.0,0.531544,1400624.0,310818.0,0.221914,0.15,0.246869,0.024955,0.101085,0.147237,rightlow_cluster
8171,4825,auto_econom,316.736312,0.200,7917659.0,3213385.0,0.405850,1400624.0,177727.0,0.126891,0.2,0.158909,0.032017,0.201483,0.076425,rightlow_cluster
8172,4825,auto_econom,316.736312,0.250,7917659.0,2683753.0,0.338958,1400624.0,122732.0,0.087627,0.25,0.107707,0.020080,0.186435,0.082382,rightlow_cluster
8173,4825,auto_econom,316.736312,0.300,7917659.0,2145782.0,0.271012,1400624.0,84542.0,0.060360,0.3,0.075302,0.014941,0.198421,0.066197,rightlow_cluster
15416,5957,auto_econom,454.014028,0.100,6753095.0,2500180.0,0.370227,986776.0,186249.0,0.188745,0.1,0.192479,0.003734,0.019402,0.019644,rightlow_cluster
15417,5957,auto_econom,454.014028,0.125,6753095.0,1936053.0,0.286691,986776.0,133031.0,0.134814,0.125,0.137345,0.002531,0.018430,0.058330,rightlow_cluster


In [407]:
rightlow_cluster

,city_id,order_type,avg_rta,alpha_value,bids_count,bad_bids_count,bad_bids_share,rides_count,rides_badbids_count,rides_badbids_share,alpha_value_str,ols_predicted_value,ols_diff,rel_diff,min_distance_to_highlight,highlight_status
3800,4208,auto_econom_dublicate,361.098065,0.100,3211709.0,753594.0,0.234640,666460.0,68028.0,1.020736e-01,0.1,0.121988,0.019914,0.163248,0.017606,rightlow_cluster
3801,4208,auto_econom_dublicate,361.098065,0.125,3211709.0,645938.0,0.201120,666460.0,55103.0,8.268013e-02,0.125,0.096350,0.013670,0.141880,0.023012,rightlow_cluster
3802,4208,auto_econom_dublicate,361.098065,0.150,3211709.0,574329.0,0.178823,666460.0,47277.0,7.093749e-02,0.15,0.083052,0.012115,0.145868,0.047647,rightlow_cluster
3803,4208,auto_econom_dublicate,361.098065,0.200,3211709.0,292662.0,0.091123,666460.0,14957.0,2.244246e-02,0.2,0.035679,0.013236,0.370989,0.037184,rightlow_cluster
3804,4208,auto_econom_dublicate,361.098065,0.250,3211709.0,211328.0,0.065799,666460.0,9572.0,1.436245e-02,0.25,0.020908,0.006546,0.313074,0.042428,rightlow_cluster
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15443,5957,moto_econom,354.015675,0.200,19183831.0,404.0,0.000021,4257602.0,3.0,7.046220e-07,0.2,0.000008,0.000008,0.914547,0.025961,rightlow_cluster
15444,5957,moto_econom,354.015675,0.250,19183831.0,391.0,0.000020,4257602.0,3.0,7.046220e-07,0.25,0.000006,0.000006,0.891203,0.025961,rightlow_cluster
15445,5957,moto_econom,354.015675,0.300,19183831.0,384.0,0.000020,4257602.0,3.0,7.046220e-07,0.3,0.000006,0.000005,0.873309,0.025962,rightlow_cluster
15446,5957,moto_econom,354.015675,0.350,19183831.0,375.0,0.000020,4257602.0,3.0,7.046220e-07,0.35,0.000004,0.000003,0.831607,0.025962,rightlow_cluster


In [408]:
ok_types_list = ['auto_econom', 'auto_econom_without_option_4_pass', 'auto_econom_karaganda', 'Ride A/C', 
                 'auto_econom_dublicate', 'auto_econom_nocomment', 'auto_econom_wheelchair', 'auto_econom_Ukraine']

recife_cluster = df_filtered[df_filtered['highlight_status'] == 'recife_cluster']
recife_cluster_refined = recife_cluster[(recife_cluster['order_type'].isin(ok_types_list)) &
                                      (recife_cluster['bad_bids_share'] >= 0.2) &
                                      (recife_cluster['ols_diff'] >= 0.0)] 

middle_cluster = df_filtered[df_filtered['highlight_status'] == 'middle_cluster']
middle_cluster_refined = middle_cluster[(middle_cluster['order_type'].isin(ok_types_list)) &
                                      (middle_cluster['bad_bids_share'] >= 0.2) &
                                      (middle_cluster['ols_diff'] >= 0.0)] 

rightlow_cluster = df_filtered[df_filtered['highlight_status'] == 'rightlow_cluster']
rightlow_cluster_refined = rightlow_cluster[(rightlow_cluster['order_type'].isin(ok_types_list)) &
                                      (rightlow_cluster['bad_bids_share'] >= 0.2) &
                                      (rightlow_cluster['ols_diff'] >= 0.0)] 

In [339]:
v2_final_list = pd.concat([
    recife_cluster_refined.sort_values('alpha_value', ascending=False).drop_duplicates(subset=['city_id'], keep='first'),
    middle_cluster_refined.sort_values('alpha_value', ascending=False).drop_duplicates(subset=['city_id'], keep='first'),
    rightlow_cluster_refined.sort_values('alpha_value', ascending=False).drop_duplicates(subset=['city_id'], keep='first')
], ignore_index=True)
v2_final_list

,city_id,order_type,avg_rta,alpha_value,bids_count,bad_bids_count,bad_bids_share,rides_count,rides_badbids_count,rides_badbids_share,alpha_value_str,ols_predicted_value,ols_diff,rel_diff,min_distance_to_highlight,highlight_status
0,5504,auto_econom,208.292639,0.150,2315121.0,591289.0,0.255403,671074.0,64447.0,0.096036,0.15,0.118619,0.022583,0.190383,0.038226,recife_cluster
1,5548,auto_econom,270.392303,0.150,4609340.0,1045408.0,0.226802,1348288.0,100764.0,0.074735,0.15,0.105335,0.030600,0.290505,0.026105,recife_cluster
2,4243,auto_econom,222.210329,0.200,9073835.0,2323276.0,0.256041,2320708.0,173034.0,0.074561,0.2,0.100252,0.025691,0.256263,0.045834,middle_cluster
3,262,auto_econom,208.435669,0.150,1216634.0,303646.0,0.249579,314198.0,34024.0,0.108288,0.15,0.115914,0.007625,0.065783,0.033580,middle_cluster
4,4825,auto_econom,316.736312,0.300,7917659.0,2145782.0,0.271012,1400624.0,84542.0,0.060360,0.3,0.075302,0.014941,0.198421,0.066197,rightlow_cluster
5,5957,auto_econom,454.014028,0.125,6753095.0,1936053.0,0.286691,986776.0,133031.0,0.134814,0.125,0.137345,0.002531,0.018430,0.058330,rightlow_cluster


In [399]:
del recife_cluster_refined, middle_cluster_refined, rightlow_cluster_refined

In [331]:
# final_list = [
#     {'city_id': 262, 'alpha_value_numeric': 0.15},
#     {'city_id': 4240, 'alpha_value_numeric': 0.15},
#     {'city_id': 5504, 'alpha_value_numeric': 0.15},
#     {'city_id': 4258, 'alpha_value_numeric': 0.20},
# ]

# final_list_df = pd.DataFrame(final_list)
# final_list_df['alpha_value_str'] = final_list_df['alpha_value_numeric'].astype(str)
# filter_final_list_df_for_merge = final_list_df[['city_id', 'alpha_value_str']]
# selected_rows_df = pd.merge(df_filtered, filter_final_list_df_for_merge, on=['city_id', 'alpha_value_str'], how='inner')
# selected_rows_df

# RES

In [ ]:
final_list = pd.concat([
    selected_rows_df_veracrus,
    selected_rows_df_puerto,
    v2_final_list
], ignore_index=True)
final_list = final_list.sort_values('city_id', ascending=False)

# final_list.to_csv('final_list.csv', index=False)
final_list

,city_id,order_type,avg_rta,alpha_value,bids_count,bad_bids_count,bad_bids_share,rides_count,rides_badbids_count,rides_badbids_share,alpha_value_str,ols_predicted_value,ols_diff,rel_diff,min_distance_to_highlight,highlight_status
15,5957,auto_econom,454.014028,0.125,6753095.0,1936053.0,0.286691,986776.0,133031.0,0.134814,0.125,0.137345,0.002531,0.018430,0.058330,rightlow_cluster
2,5548,auto_econom,270.392303,0.200,4609340.0,692657.0,0.150272,1348288.0,51635.0,0.038297,0.2,0.058838,0.020542,0.349121,0.052559,veracrus_cluster
11,5548,auto_econom,270.392303,0.150,4609340.0,1045408.0,0.226802,1348288.0,100764.0,0.074735,0.15,0.105335,0.030600,0.290505,0.026105,recife_cluster
1,5504,auto_econom,208.292639,0.200,2315121.0,362965.0,0.156780,671074.0,34667.0,0.051659,0.2,0.061387,0.009728,0.158464,0.047049,veracrus_cluster
9,5504,auto_econom,208.292639,0.150,2315121.0,591289.0,0.255403,671074.0,64447.0,0.096036,0.15,0.118619,0.022583,0.190383,0.038226,puerto_cluster
10,5504,auto_econom,208.292639,0.150,2315121.0,591289.0,0.255403,671074.0,64447.0,0.096036,0.15,0.118619,0.022583,0.190383,0.038226,recife_cluster
14,4825,auto_econom,316.736312,0.300,7917659.0,2145782.0,0.271012,1400624.0,84542.0,0.060360,0.3,0.075302,0.014941,0.198421,0.066197,rightlow_cluster
0,4271,auto_econom,200.416556,0.150,1901412.0,253299.0,0.133216,594691.0,26782.0,0.045035,0.15,0.061871,0.016835,0.272106,0.039107,veracrus_cluster
8,4258,auto_econom,171.632032,0.200,6095409.0,1849051.0,0.303351,1519645.0,174811.0,0.115034,0.2,0.118776,0.003742,0.031502,0.062459,puerto_cluster
12,4243,auto_econom,222.210329,0.200,9073835.0,2323276.0,0.256041,2320708.0,173034.0,0.074561,0.2,0.100252,0.025691,0.256263,0.045834,middle_cluster


In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project='analytics-dev-333113')

query = f'''
SELECT DISTINCT tc.id AS city_id, 
                -- CONCAT(tc.name, ' (', tc.id, ')') AS city, 
                  tc.name AS city, 
                  CONCAT(tcc.name) AS country
             FROM `indriver-e6e40.ods_geo_config.tbl_city` tc
             JOIN `indriver-e6e40.ods_geo_config.tbl_country` tcc 
             ON tc.country_id = tcc.id'''

names_df = client.query(query).result().to_dataframe()
# names_df

final_list_with_names = pd.merge(names_df, final_list, on='city_id', how='right')
final_list_with_names = final_list_with_names[['highlight_status'] + 
                                              [col_name for col_name in  list(final_list_with_names.columns) if col_name != 'highlight_status']]
final_list_with_names = final_list_with_names.drop(['alpha_value_str', 'ols_predicted_value', 'ols_diff', 'rel_diff', 'min_distance_to_highlight'], axis=1)
final_list_with_names = final_list_with_names.sort_values('highlight_status', ascending=False).reset_index(drop=True)
final_list_with_names.to_csv('final_list.csv', index=False)
final_list_with_names

In [411]:
from google.cloud import bigquery

client = bigquery.Client(project='analytics-dev-333113')

query = f'''
SELECT DISTINCT tc.id AS city_id, 
                -- CONCAT(tc.name, ' (', tc.id, ')') AS city, 
                  tc.name AS city, 
                  CONCAT(tcc.name) AS country
             FROM `indriver-e6e40.ods_geo_config.tbl_city` tc
             JOIN `indriver-e6e40.ods_geo_config.tbl_country` tcc 
             ON tc.country_id = tcc.id'''

names_df = client.query(query).result().to_dataframe()
# names_df

middle_rl_list = pd.concat([
    middle_cluster_refined.sort_values('alpha_value', ascending=False).drop_duplicates(subset=['city_id'], keep='first'),
    rightlow_cluster_refined.sort_values('alpha_value', ascending=False).drop_duplicates(subset=['city_id'], keep='first')
], ignore_index=True)

middle_rl_with_names = pd.merge(names_df, middle_rl_list, on='city_id', how='right')
middle_rl_with_names = middle_rl_with_names[['highlight_status'] + 
                                              [col_name for col_name in  list(middle_rl_with_names.columns) if col_name != 'highlight_status']]
middle_rl_with_names = middle_rl_with_names.drop(['alpha_value_str', 'ols_predicted_value', 'ols_diff', 'rel_diff', 'min_distance_to_highlight'], axis=1)
middle_rl_with_names = middle_rl_with_names.sort_values('highlight_status', ascending=False).reset_index(drop=True)
middle_rl_with_names.to_csv('middle_rl_clusters.csv', index=False)
middle_rl_with_names

,highlight_status,city_id,city,country,order_type,avg_rta,alpha_value,bids_count,bad_bids_count,bad_bids_share,rides_count,rides_badbids_count,rides_badbids_share
0,rightlow_cluster,4825,Santo Domingo,Dominican Republic,auto_econom,316.736312,0.300,7917659.0,2145782.0,0.271012,1400624.0,84542.0,0.060360
1,rightlow_cluster,4208,Rabat (الرباط),Morocco,auto_econom_dublicate,361.098065,0.125,3211709.0,645938.0,0.201120,666460.0,55103.0,0.082680
2,rightlow_cluster,5957,Kathmandu,Nepal,auto_econom,454.014028,0.125,6753095.0,1936053.0,0.286691,986776.0,133031.0,0.134814
3,middle_cluster,4242,Medellin,Colombia,auto_econom,258.999842,0.200,13480635.0,3423474.0,0.253955,3055928.0,213072.0,0.069724
4,middle_cluster,4404,Quito,Ecuador,auto_econom,247.450454,0.200,7850015.0,1851513.0,0.235861,1632350.0,98541.0,0.060368
5,middle_cluster,4243,Barranquilla,Colombia,auto_econom,222.210329,0.200,9073835.0,2323276.0,0.256041,2320708.0,173034.0,0.074561
6,middle_cluster,238,Уральск,Казахстан,auto_econom_wheelchair,211.082248,0.200,3090121.0,881582.0,0.285290,678280.0,69574.0,0.102574
7,middle_cluster,4252,Pereira,Colombia,auto_econom,291.902413,0.200,2130203.0,449557.0,0.211040,578916.0,26038.0,0.044977
8,middle_cluster,262,Павлодар,Казахстан,auto_econom,208.435669,0.150,1216634.0,303646.0,0.249579,314198.0,34024.0,0.108288
9,middle_cluster,4229,Leon de los Aldama,Mexico,auto_econom,335.369288,0.150,1700357.0,388740.0,0.228623,421177.0,40344.0,0.095789
